# Retrieval-Augmented Generation: Question Answering based on Custom Dataset

Many use cases such as building a chatbot require text (text2text) generation models like **[BloomZ 7B1](https://huggingface.co/bigscience/bloomz-7b1)**, **[Flan T5 XXL](https://huggingface.co/google/flan-t5-xxl)**, and **[Flan T5 UL2](https://huggingface.co/google/flan-ul2)** to respond to user questions with insightful answers. The **BloomZ 7B1**, **Flan T5 XXL**, and **Flan T5 UL2** models have picked up a lot of general knowledge in training, but we often need to ingest and use a large library of more specific information.

<img src="rag.png" max-width="1080"/>

Retrieval-Augmented Generation (RAG) combines the power of pre-trained LLMs with information retrieval - enabling more accurate and context-aware responses. 

1. The first process in a chat bot is to generate embeddings. Typically you will have an ingestion process which will run through your embedding model and generate the embeddings which will be stored in a sort of a vector store as your knowledge base. 

2. Second process is to retrieve relevant information from the knowledge base, and generate a response based on retrieved information and input query.

In this notebook we will demonstrate how to use **AI21 Contextual Answer** to answer questions using a library of documents as a reference, by using document embeddings and retrieval. The embeddings are generated from [all-mpnet-base-v2](https://huggingface.co/sentence-transformers/all-mpnet-base-v2), a [sentence-transformers](https://www.sbert.net/) embedding model.

**This notebook serves a template such that you can easily replace the example dataset by your own to build a custom question and asnwering application.**

This notebook is tested with SageMaker Studio with following configuration:
- Image: PyTorch 1.13 Python 3.9 CPU Optimized
- Kernel: Python 3
- Instance type: ml.t3.medium

## Step 1. Setup

To better illustrate the idea, let's first libraries and functions that are required to perform the demo. The AI21 LLM has been pre-deployed before, but if you want to look at example of deploying LLM to SageMaker you can look at [rag_falcon]('./rag_falcon.ipynb') notebook.

In [1]:
!pip install --upgrade sagemaker --quiet
!pip install ipywidgets==7.0.0 --quiet


[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import time
import sagemaker, boto3, json
from sagemaker.session import Session
from sagemaker.model import Model
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
model_version = "*"

In [3]:
url = "https://ep928kbfdd.execute-api.us-east-1.amazonaws.com/test/"

In [4]:
import requests

def query_endpoint_with_json_payload(url, payload):
    response = requests.post(
        url,
        json=payload,
    )
    return response

def parse_response(query_response):
    model_predictions = query_response.json()
    generated_text = model_predictions["answer"]
    return generated_text

## Step 2. Ask a question to LLM without providing the context

To better illustrate why we need retrieval-augmented generation (RAG) based approach to solve the question and anwering problem. Let's directly ask the model a question and see how they respond.

In [5]:
payload = {
    "context": "",
    "question": "Which instances can I use with Managed Spot Training in SageMaker?"
}

response = query_endpoint_with_json_payload(url, payload)
response = parse_response(response)
print(response)

Answer not in document


## Step 3. Improve the answer to the same question using prompt engineering with insightful context

To better answer the question well, we provide extra contextual information, combine it with a prompt, and send it to model together with the question. Below is an example.

In [6]:
payload = {
    "context": "Managed Spot Training can be used with all instances supported in Amazon SageMaker. Managed Spot Training is supported in all AWS Regions where Amazon SageMaker is currently available.",
    "question": "Which instances can I use with Managed Spot Training in SageMaker?"
}

response = query_endpoint_with_json_payload(url, payload)
response = parse_response(response)
print(response)

All instances supported by Amazon SageMaker can be used with Managed Spot Training.


The output is already significantly better than asking without providing any context. 

Now, the question becomes where can I find the insightful context based on the user query? The answer is to use a pre-stored knowledge data base with retrieval augmented generation, as shown below.

## Step 4. Use RAG based approach to identify the correct documents, and use them and question to query LLM

We plan to use document embeddings to fetch the most relevant documents in our document knowledge library and combine them with the prompt that we provide to LLM.

To achieve that, we will do following.

- Generate embedings for each of document in the knowledge library with the HuggingFace sentence-transformers embedding model.
- Identify top K most relevant documents based on user query.
  - For a query of your interest, generate the embedding of the query using the same embedding model.
  - Search the indexes of top K most relevant documents in the embedding space using the SageMaker KNN algorithm.
  - Use the indexes to retrieve the corresponded documents.
- Combine the retrieved documents with prompt and question and send them into LLM.

Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt. 



### 4.1 Preparing the embedding model

We'll start with initializing our embeddings model. In this notebook, we are using all-mpnet-base-v2 model. 

If you want, you can experiment with other embedding model, for example you can use GPT-J-6B embedding model which you can easily deploy with a few clicks from SageMaker JumpStart UI.

In [7]:
pip install -U sentence-transformers


[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import sentence_transformers

from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("all-mpnet-base-v2")

### 4.2. Generate embedings for each of document in the knowledge library with the embedding model.

For the purpose of the demo we will use [Amazon SageMaker FAQs](https://aws.amazon.com/sagemaker/faqs/) as knowledge library. The data are formatted in a CSV file with two columns Question and Answer. We use only the Answer column as the documents of knowledge library, from which relevant documents are retrieved based on a query.

Each row in the CSV format dataset corresponds to a textual document. We will iterate each document to get its embedding vector via the all-mpnet-base-v2 embedding models. For your purpose, you can replace the example dataset of your own to build a custom question and answering application.

In [9]:
import pandas as pd

df_knowledge = pd.read_csv("Amazon_SageMaker_FAQs.csv", header=None, usecols=[1], names=["Answer"])
df_knowledge.head(5)

,Answer
0,Amazon SageMaker is a fully managed service to...
1,For a list of the supported Amazon SageMaker A...
2,Amazon SageMaker is designed for high availabi...
3,Amazon SageMaker stores code in ML storage vol...
4,Amazon SageMaker ensures that ML model artifac...


In [10]:
vectors = encoder.encode(df_knowledge['Answer'])

In [11]:
vectors.shape[1]

768

## 4.3. Index the embedding knowledge library

We will store and match the embeddings using a vector database. In this notebook, we will showcase [FAISS](https://github.com/facebookresearch/faiss) which will be transient and in memory.

In real-world scenarios, [Amazon Opensearch](https://aws.amazon.com/opensearch-service/) is a popular choice for vector DB. You may refer to this [blog](https://aws.amazon.com/blogs/machine-learning/build-a-powerful-question-answering-bot-with-amazon-sagemaker-amazon-opensearch-service-streamlit-and-langchain/) and follow the instructions to build your own RAG solution with Opensearch! Alternatively, you may also use [Amazon Kendra](https://aws.amazon.com/kendra/) for its built-in NLP capabilities and pre-trained domain knowledge. Refer to this [blog](https://aws.amazon.com/blogs/machine-learning/quickly-build-high-accuracy-generative-ai-applications-on-enterprise-data-using-amazon-kendra-langchain-and-large-language-models/) for guided instructions on how to set it up.

In [12]:
pip install faiss-cpu


[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
import faiss

vector_dimension = vectors.shape[1]
index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(vectors)
index.add(vectors)

### 4.4 Retrieve the most relevant documents

Given the embedding of a query, we will query the endpoint to get the indexes of top K most relevant documents and use the indexes to retrieve the corresponded textual documents.

Next, the textual documents are concatenated with maximum length of MAX_SECTION_LEN. This is to make sure the context we send into the prompt contains a good enough amount of information all the while not exceeding model's capacity.

In [14]:
MAX_SECTION_LEN = 1500
THRESHOLD = 0.5
SEPARATOR = "\n* "


def construct_context(
    context_predictions_idx, 
    context_prediction_dist, 
    df_knowledge, 
    threshold, 
    context_length
) -> str:
    chosen_sections = []
    chosen_sections_len = 0

    for index, dist in zip(context_predictions_idx, context_prediction_dist):
        document_section = df_knowledge.loc[index]
        chosen_sections_len += len(document_section) + 2
        if dist > threshold:
            break
        chosen_sections.append(SEPARATOR + document_section.replace("\n", " "))
    
    concatenated_doc = "".join(chosen_sections)[:context_length]
    print(
        f"With maximum sequence length {MAX_SECTION_LEN}, \
selected top {len(chosen_sections)} document sections: {concatenated_doc}"
    )

    return concatenated_doc, len(chosen_sections)

Now we can convert question to embedding, and search for the most relevant documents. In this example, we will retrieve top 5 most relevant documents (k=5).

In [15]:
import numpy as np

question = 'Which instances can I use with Managed Spot Training in SageMaker?'
search_vector = encoder.encode(question)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

In [16]:
k = 5
distances, indices = index.search(_vector, k=k)
distances, indices

(array([[0.23976025, 0.35568824, 0.4087937 , 0.51908976, 0.5701388 ]],
       dtype=float32),
 array([[90, 84, 91, 87, 85]]))

In [17]:
context_retrieve, num_context_doc = construct_context(
    indices[0], 
    distances[0], 
    df_knowledge["Answer"], 
    threshold=THRESHOLD,
    context_length=MAX_SECTION_LEN
)

With maximum sequence length 1500, selected top 3 document sections: 
  Managed Spot Training can be used with all instances supported in Amazon SageMaker.
* Managed Spot Training with Amazon SageMaker lets you train your ML models using Amazon EC2 Spot instances, while reducing the cost of training your models by up to 90%.
  Managed Spot Training is supported in all AWS Regions where Amazon SageMaker is currently available.


### 4.5 Combine the retrieved documents, prompt, and question to query the LLM

In [18]:
context = context_retrieve

payload = {
    "context": context_retrieve,
    "question": question
}

response = query_endpoint_with_json_payload(url, payload)
response = parse_response(response)
print(response)

Managed Spot Training can be used with all instances supported in Amazon SageMaker.


## Bonus

**AI21 Studio Contextual Answers** model allows you to access our high-quality question answering technology. It was designed to answer questions based on a specific document context provided by the customer. This avoids any factual issues that language models may have and makes sure the answers it provides are grounded in that context document.

This model receives document text, serving as a context, and a question and returns an answer based entirely on this context. This means that if the answer to your question is not in the document, the model will indicate it (instead of providing a false answer).

To get a sense of the model's behavior, let's use this toy example of asking what is the Eiffel tower height. Most language models will simply answer according to their training data.

This model, however, bases its answer solely on the context you provide. Let's use the following [Wikipedia article](https://en.wikipedia.org/wiki/Eiffel_Tower) as context, with small modifications:

In [20]:
# Actual paragraph
context = "The tower is 330 metres (1,083 ft) tall,[6] about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest human-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure in the world to surpass both the 200-metre and 300-metre mark in height. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."

# The paragraph with manual changes of the height
false_context = "The tower is 3 metres (10 ft) tall,[6] about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest human-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure in the world to surpass both the 200-metre and 300-metre mark in height. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."

# The paragraph with the height omitted
partial_context = "Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest human-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure in the world to surpass both the 200-metre and 300-metre mark in height. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."

In [21]:
# True context
payload = {
    "context": context,
    "question": "What is the height of the Eiffel tower?"
}
query_endpoint_with_json_payload(url, payload).json()


{'answer': 'The Eiffel Tower is 330 metres tall.'}

In [23]:
# False context
payload = {
    "context": false_context,
    "question": "What is the height of the Eiffel tower?"
}
query_endpoint_with_json_payload(url, payload).json()

{'answer': 'The Eiffel Tower is 3 metres (10 ft) tall.'}

In [25]:
# Partial context
payload = {
    "context": partial_context,
    "question": "What is the height of the Eiffel tower?"
}
query_endpoint_with_json_payload(url, payload).json()

{'answer': 'excluding transmitters, the Eiffel Tower is now taller than the Chrysler Building by 5.2 metres (17 ft).'}

### Ask about financial reports

The document context should be **no more than 10,000 characters**, and the question can be up to 160 characters.

Imagine you are performing research and rely on financial reports to base your findings. Let's take the following part from [JPMorgan Chase & Co. 2021 annual report](https://www.jpmorganchase.com/content/dam/jpmc/jpmorgan-chase-and-co/investor-relations/documents/annualreport-2021.pdf):

In [26]:
financial_context = """In 2020 and 2021, enormous QE — approximately $4.4 trillion, or 18%, of 2021 gross domestic product (GDP) — and enormous fiscal stimulus (which has been and always will be inflationary) — approximately $5 trillion, or 21%, of 2021 GDP — stabilized markets and allowed companies to raise enormous amounts of capital. In addition, this infusion of capital saved many small businesses and put more than $2.5 trillion in the hands of consumers and almost $1 trillion into state and local coffers. These actions led to a rapid decline in unemployment, dropping from 15% to under 4% in 20 months — the magnitude and speed of which were both unprecedented. Additionally, the economy grew 7% in 2021 despite the arrival of the Delta and Omicron variants and the global supply chain shortages, which were largely fueled by the dramatic upswing in consumer spending and the shift in that spend from services to goods. Fortunately, during these two years, vaccines for COVID-19 were also rapidly developed and distributed.
In today's economy, the consumer is in excellent financial shape (on average), with leverage among the lowest on record, excellent mortgage underwriting (even though we've had home price appreciation), plentiful jobs with wage increases and more than $2 trillion in excess savings, mostly due to government stimulus. Most consumers and companies (and states) are still flush with the money generated in 2020 and 2021, with consumer spending over the last several months 12% above pre-COVID-19 levels. (But we must recognize that the account balances in lower-income households, smaller to begin with, are going down faster and that income for those households is not keeping pace with rising inflation.)
Today's economic landscape is completely different from the 2008 financial crisis when the consumer was extraordinarily overleveraged, as was the financial system as a whole — from banks and investment banks to shadow banks, hedge funds, private equity, Fannie Mae and many other entities. In addition, home price appreciation, fed by bad underwriting and leverage in the mortgage system, led to excessive speculation, which was missed by virtually everyone — eventually leading to nearly $1 trillion in actual losses.
"""

Rather than reading the entire report, just ask what you want to know. The model will answer based on the provided report:


In [27]:
payload = {
    "context": financial_context,
    "question": "Did the economy shrink after the Omicron variant arrived?"
}
query_endpoint_with_json_payload(url, payload).json()

{'answer': 'No, the economy grew 7% in 2021 despite the arrival of the Delta and Omicron variants and the global supply chain shortages'}

In addition, you can ask more complex questions, where the answer requires deductions rather than just extracting the correct sentence from the document context. This will result in abstractive, rather than extractive, answers that draw on several different parts of the document. For example, look at the following question:

In [28]:
payload = {
    "context": financial_context,
    "question":  "Did COVID-19 eventually help the economy?"
}
query_endpoint_with_json_payload(url, payload).json()

{'answer': 'The arrival of COVID-19 vaccines in early 2021 helped boost the economy. In addition, the dramatic upswing in consumer spending and the shift in that spend from services to goods largely fueled the 7% GDP growth in 2021.'}

We now present the model with the following question. You may be confused to answer something based on the last paragraph without delving into the text. However, if you read the provided document context properly, you will discover that the answer does not appear there. The model handles this as expected:

In [29]:
payload = {
    "context": financial_context,
    "question":  "How did COVID-19 affect the financial crisis of 2008?"
}
query_endpoint_with_json_payload(url, payload).json()

{'answer': 'Answer not in document'}